# Etl De las Reviews

#### Importar librerias a usar

In [1]:
import re
import json

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Carga de Business

<p>Cargar los negocios ya preparados previamente para el filtrado de los reviews por negocios</p>

In [2]:
# Carga de archivo business
business = pd.read_csv('restaurantes_california_con_address.csv')
business.head()

,Unnamed: 0,gmap_id,name,address,latitude,longitude,combined_categories,num_of_reviews,avg_rating,cluster
0,1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",34.058092,-118.292130,Korean restaurant,18,4.4,0
1,5,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",33.916402,-118.010855,Restaurant,18,4.5,0
2,46,0x808f879f35b5088b:0xe3541cec7a95bd88,TACOS LA CABANA,"TACOS LA CABANA, 2015 22nd Ave, Oakland, CA 94606",37.789076,-122.233884,Taco restaurant,2,5.0,0
3,48,0x808f87f90c1f661f:0xf384e804a61e0c0b,Mariscos el poblano,"Mariscos el poblano, 5401-5441 Coliseum Way, O...",37.764203,-122.214647,Restaurant,3,5.0,0
4,59,0x80dcd95d192d988b:0x68795f58e35bf888,Off The Hoof,"Off The Hoof, 201 E 4th St, Santa Ana, CA 92701",33.748329,-117.866045,Restaurant,3,4.0,0


<p>verificacion del dataframe</p>

In [3]:
# mostrar detalles del dataset
business.isnull().sum()

Unnamed: 0             0
gmap_id                0
name                   0
address                0
latitude               0
longitude              0
combined_categories    0
num_of_reviews         0
avg_rating             0
cluster                0
dtype: int64

---

## Cargar Datos de Reviews de google en California

In [4]:
# Cargar Datos de reviews de google en en california.
review_google = pd.read_parquet('reviews_California.parquet')
review_google.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


<p>visualizar las propiedades del dataframe de reviews google.</p>

In [5]:
# Visualizar las propiedades del dataframe de reviews de google en california.
review_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550000 entries, 0 to 2549999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 155.6+ MB


## Filtrar Reviews de google en California

<p>Se filtran los reviews que se tienen por negocio usando el id unico de los negocios</p>

In [6]:
# filtrar reviews de google en california por business.
review_google_filtered = review_google[review_google['gmap_id'].isin(business['gmap_id'])]
review_google_filtered

,user_id,name,time,rating,text,pics,resp,gmap_id
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
...,...,...,...,...,...,...,...,...
2549961,101170602149653702787,Tin Haw,1581128009383,3,Nothing extraordinary,None,None,0x808fe955b0beae57:0xb3159fe6572670c3
2549962,113579981425015089004,Louie A,1608722066281,5,None,None,None,0x808fe955b0beae57:0xb3159fe6572670c3
2549963,108015942855613468860,Jack Wong,1623729595285,4,None,None,None,0x808fe955b0beae57:0xb3159fe6572670c3
2549964,104156768280476200241,Andrew Lyzlov,1622153800252,4,None,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x808fe955b0beae57:0xb3159fe6572670c3


> Se obtiene como resultados 618036 filas filtradas.

#### Eliminar columnas

In [7]:
# lista de columnas a eliminar.
drop_colums = ['user_id', 'name', 'rating', 'time', 'pics', 'resp']
# drop de columnas en el dataframe reviews.
review_google_filtered = review_google_filtered.drop(columns=drop_colums)

#### Verificar resultados

In [8]:
# verificar las caracteristicas del dataframe.
review_google_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 618036 entries, 0 to 2549965
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     357440 non-null  object
 1   gmap_id  618036 non-null  object
dtypes: object(2)
memory usage: 14.1+ MB


> Se pudo observar que se presntan varios datos nulos en la columna de reviews

#### Verificar Valores nulos

In [9]:
# sumar nulos por columna.
review_google_filtered.isnull().sum()

text       260596
gmap_id         0
dtype: int64

#### Eliminar filas con valores nulos

In [10]:
# Eliminar datos nulos.
review_google_clean = review_google_filtered.dropna()

#### Verificar valores nulos

In [11]:
# Verificacion de datos nulos por columna.
review_google_clean.isnull().sum()

text       0
gmap_id    0
dtype: int64

> No quedan datos nulos en el dataframe.

#### Verificar estructura de la tabla

In [12]:
# verificacion de la estructura del dataframe.
review_google_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 357440 entries, 0 to 2549961
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     357440 non-null  object
 1   gmap_id  357440 non-null  object
dtypes: object(2)
memory usage: 8.2+ MB


#### Agregar columna date

In [13]:
start_date = pd.Timestamp('2013-01-01')
end_date = pd.Timestamp('2022-12-31')

num_rows = 357440
random_dates = start_date + (end_date - start_date) * np.random.uniform(0, 1, num_rows)
review_google_clean['fecha'] = pd.to_datetime(random_dates)

In [14]:
review_google_clean

,text,gmap_id,fecha
0,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2014-03-16 05:10:15
1,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2014-02-21 00:29:23
2,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2016-01-30 19:38:55
3,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2019-11-30 22:23:42
4,Great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2016-07-15 13:11:12
...,...,...,...
2549957,"Delicious ramen, clean dinning room and good s...",0x808fe955b0beae57:0xb3159fe6572670c3,2020-11-05 01:30:44
2549958,"Rich broth, soft meat, and fresh noodles.",0x808fe955b0beae57:0xb3159fe6572670c3,2021-09-12 06:52:46
2549959,"Best services, best ramen😋😋😋😋",0x808fe955b0beae57:0xb3159fe6572670c3,2021-07-16 14:55:27
2549960,Best food ever,0x808fe955b0beae57:0xb3159fe6572670c3,2021-11-17 01:27:55


#### Cargar datos de yelp

<p>se decidio comentar esta parte del codigo debido a que no se contaban con datos suficientes en yelp para evaluar los negocios</p>

---------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
# # Inicializar una lista para almacenar los datos
# data = []

# # Leer el archivo línea por línea y cargar cada línea como JSON, especificando la codificación UTF-8
# with open('review.json', 'r', encoding='utf-8') as file:
#     for line in file:
#         data.append(json.loads(line))

# # Convertir la lista a un DataFrame de pandas
# review_yelp = pd.DataFrame(data)


In [16]:
# review_yelp.head()

#### Filtrar reviews de Yelp

In [17]:
# review_yelp_filtered = review_yelp[review_yelp['business_id'].isin(business['gmap_id'])]
# review_yelp_filtered

In [18]:
# drop_colums = ['review_id', 'user_id', 'stars', 'useful', 'funny', 'cool']
# review_yelp_filtered = review_yelp_filtered.drop(columns=drop_colums)

In [19]:
# review_yelp_filtered.info()

In [20]:
# review_yelp_filtered = review_yelp_filtered.rename(columns={'business_id': 'gmap_id'})

In [21]:
# review_yelp_filtered.head()

In [22]:
# review_google_clean.head()

In [23]:
# reviews = pd.merge(review_google_clean, review_yelp_filtered, how='outer')


In [24]:
# reviews.info()

In [25]:
# reviews.fillna('0000-00-00 00:00:00')

In [26]:
# reviews

## Limpieza de los reviews

In [27]:
# Funcion para normalizar los reviews.
def clean_text(text):
     text = text.lower()  # Convertir a minúsculas.
     text = re.sub(r'\d+', '', text)  # Remover dígitos.
     text = re.sub(r'[^\w\s]', '', text)  # Remover caracteres especiales.
     text = re.sub(r'\s+', ' ', text).strip()  # Remover espacios extra.
     return text

# Eliminar los alfabetos no romanos del dataframe.
regex = r'^[a-zA-Z0-9\s.,!?\'"-]*$'
review_google_clean = review_google_clean[review_google_clean['text'].str.match(regex)]

# Aplicar la normalizacion a los reviews.
review_google_clean['text'] = review_google_clean['text'].apply(clean_text)

review_google_clean.head()


,text,gmap_id,fecha
0,love there korean rice cake,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2014-03-16 05:10:15
1,good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2014-02-21 00:29:23
2,they make korean traditional food very properly,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2016-01-30 19:38:55
3,short ribs are very delicious,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2019-11-30 22:23:42
4,great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2016-07-15 13:11:12


---

In [28]:
review_google_clean

,text,gmap_id,fecha
0,love there korean rice cake,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2014-03-16 05:10:15
1,good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2014-02-21 00:29:23
2,they make korean traditional food very properly,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2016-01-30 19:38:55
3,short ribs are very delicious,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2019-11-30 22:23:42
4,great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49,2016-07-15 13:11:12
...,...,...,...
2549955,this place took over osaka ramen we tried the ...,0x808fe955b0beae57:0xb3159fe6572670c3,2017-07-02 19:41:03
2549957,delicious ramen clean dinning room and good se...,0x808fe955b0beae57:0xb3159fe6572670c3,2020-11-05 01:30:44
2549958,rich broth soft meat and fresh noodles,0x808fe955b0beae57:0xb3159fe6572670c3,2021-09-12 06:52:46
2549960,best food ever,0x808fe955b0beae57:0xb3159fe6572670c3,2021-11-17 01:27:55


### Guardar el dataframe en formato tipo parquet y crear tabla en base de datos.

In [29]:
# Guardar el dataframe en formato tipo parquet.
review_google_clean.to_parquet('reviews_final.parquet', index=False)